In [1]:
import torch
from lm_from_scratch.corpus.decision_corpus import DecisionCorpus
from lm_from_scratch.tokenizers.char_tokenizer import CharLevelTokenizer
from lm_from_scratch.models.gpt_sa import Gpt

BATCH_SIZE = 32 # how many independent sequences will we process in parallel?
BLOCK_SIZE = 8 # what is the maximum context length for predictions?
MAX_ITERS = 10000
EVAL_INTERVAL = 1000
LEARNING_RATE = 1e-3
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EVAL_ITERS = 200

N_EMBEDDING = 32
HEAD_SIZE = 32

In [2]:
corpus = DecisionCorpus()

corpus_df = corpus.df.sample(
        frac=1,
        random_state=42
    ).reset_index(
        drop=True)

# Train and test splits
n = int(0.9*len(corpus_df)) # first 90% will be train, rest val

data = corpus.get_text()

tokenizer = CharLevelTokenizer("".join(data))
corpus_df["idx"] = [torch.tensor(tokenizer.encode(text), dtype=torch.long) for text in data]


train_data = corpus_df["idx"][:n]
val_data = corpus_df["idx"][n:].reset_index(drop=True)

In [3]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data

    doc_ix = torch.randint(len(data), (BATCH_SIZE,))
    doc_token_ix = []

    for doc_i in doc_ix:
        ix = torch.randint(len(data[int(doc_i)]) - BLOCK_SIZE, (1,))
        doc_token_ix.append((int(doc_i), ix))
    
    x = torch.stack([data[i][j:j+BLOCK_SIZE] for i,j in doc_token_ix])
    y = torch.stack([data[i][j+1:j+BLOCK_SIZE+1] for i,j in doc_token_ix])

    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y


In [4]:
model = Gpt(tokenizer.vocab_size, HEAD_SIZE, N_EMBEDDING, block_size=BLOCK_SIZE, dropout=0.1)
m = model.to(DEVICE)

In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):

    # every once in a while evaluate the loss on train and val sets
    if iter % EVAL_INTERVAL == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 5.0467, val loss 5.0479
step 1000: train loss 2.3923, val loss 2.4102
step 2000: train loss 2.2633, val loss 2.3046
step 3000: train loss 2.2288, val loss 2.2711
step 4000: train loss 2.2010, val loss 2.2440
step 5000: train loss 2.1862, val loss 2.2308
step 6000: train loss 2.1779, val loss 2.2296
step 7000: train loss 2.1748, val loss 2.2051
step 8000: train loss 2.1440, val loss 2.2112
step 9000: train loss 2.1495, val loss 2.2261
